**Assignment 4: Adverserial Search (Given: 28 Feb 2023, Due: 19 Mar 2023)**


**General instructions**

* Solutions are to be typed in the `.ipynb` file provided and uploaded in the lab course page in Moodle on the due date. 
* Your code should be well commented and should be compatible with python3.
* For this assignment, you are allowed to import the libraries `random` and `copy` of python3. No other libraries may be imported.


# Adverserial search

The Tic-Tac-Toe game starts on a 3x3 grid with two players "X" and "O" who take turns and play. The rules are as follows: each player gets a turn with player "X" (resp. "O") writing an "X" (resp. "O") in an empty cell of the grid. The game starts with the move of the "O" player. The first player to write on three horizontal or vertical or diagonal cells wins.

(a) Use the minimax strategy to design an AI that plays the game optimally. The leaf nodes where "X" wins gets 1 and "O" wins gets -1 and neither wins gets a zero. A sample game play is given below.

In [12]:
class xo:
    def __init__(self):
        self.grid=((' ',' ',' '),(' ',' ',' '),(' ',' ',' '))
        self.player_turn='O'
        self.utility=0
        self.num_turns_played=0

    def moves(self,nod):
        return [i for i in range(1,10) if nod[(i-1)//3][(i-1)%3]==' ']

    def result(self,nod,action):
        if nod.player_turn=='O':
            temp=[]
            for row in nod.state:
                temp.append(list(row))
            temp[(action-1)//3][(action-1)%3]='O'
            temp1=[]
            for row in temp:
                temp1.append(tuple(row))
            return tuple(temp1)
        
        temp=[]
        for row in nod.state:
            temp.append(list(row))
        temp[(action-1)//3][(action-1)%3]='X'
        temp1=[]
        for row in temp:
            temp1.append(tuple(row))
        return tuple(temp1)
                
    def o_play(self,index): 
        temp=[]
        for row in self.grid:
            temp.append(list(row))
        if(temp[(index-1)//3][(index-1)%3]==' '):
            temp[(index-1)//3][(index-1)%3]='O'
        else:
            self.o_play(int(input("wrong position! please enter another position: ")))
            return
        temp1=[]
        for row in temp:
            temp1.append(tuple(row))
        self.grid=tuple(temp1)
        self.player_turn='X'
        self.num_turns_played+=1

    def terminal_state(self,nod):
        if nod.player_turn=='O':
            for i in range(3):
                for j in range(3):
                    if not nod.state[i][j]=='X':
                        break
                else:
                    nod.utility=1
                    return 1
            
            for i in range(3):
                for j in range(3):
                    if not nod.state[j][i]=='X':
                        break
                else:
                    nod.utility=1
                    return 1
                
            for i in range(3):
                if not nod.state[i][i]=='X':
                    break
            else:
                nod.utility=1
                return 1
                                                                     
            for i in range(3):
                if not nod.state[i][2-i]=='X':
                    break
            else:
                nod.utility=1
                return 1
            
            return 0
                                                                     
        for i in range(3):
            for j in range(3):
                if not nod.state[i][j]=='O':
                    break
            else:
                nod.utility=-1
                return 2
        
        for i in range(3):
            for j in range(3):
                if not nod.state[j][i]=='O':
                    break
            else:
                nod.utility=-1
                return 2
            
        for i in range(3):
            if not nod.state[i][i]=='O':
                break
        else:
            nod.utility=-1
            return 2
                                                                    
        for i in range(3):
            if not nod.state[i][2-i]=='O':
                break
        else:
            nod.utility=-1
            return 2
        
        if nod.num_turns_played==9:
            return 3
        return 0
        
    def display(self):
        for i in range(2):
            for j in range(2):
                print(f"{self.grid[i][j]}|",end='')
            print(self.grid[i][2])
            print("-+-+-")
        print(f'{self.grid[2][0]}|{self.grid[2][1]}|{self.grid[2][2]}\n')
                                                                     
    def x_play(self):
        index=minmax(self,node(self.grid,self.num_turns_played,self.player_turn))
        temp=[]
        for row in self.grid:
            temp.append(list(row))
        if(temp[(index-1)//3][(index-1)%3]==' '):
            temp[(index-1)//3][(index-1)%3]='X'
        else:
            self.o_play(int(input("wrong position! please enter another position: ")))
        temp1=[]
        for row in temp:
            temp1.append(tuple(row))
        self.grid=tuple(temp1)
        self.player_turn='O'
        self.num_turns_played+=1

    def play_game(self):
        terminal_state=self.terminal_state(node(self.grid,self.num_turns_played,self.player_turn))
        while not terminal_state:
            self.display()
            if self.player_turn=='O':
                self.o_play(int(input("enter a position for O player: ")))
            else:
                self.x_play()
            terminal_state=self.terminal_state(node(self.grid,self.num_turns_played,self.player_turn))
        self.display()
        if terminal_state==1:
            print("X player wins")
            
        elif terminal_state==2:
            print("O player wins")
            
        else:
            print("Game ended in a Draw")
                
class node:
    def __init__(self,state,num_turns,to_play,action=None,utility=0):
        self.state=state
        self.utility=utility
        self.num_turns_played=num_turns    
        self.player_turn=to_play
        self.action=action

    def child_gen(self,game:xo):
        ret=[]
        for move in game.moves(self.state):
            if self.player_turn=='X':
                z='O'
            else:
                z='X'
            nod=node(game.result(self,move),self.num_turns_played+1,z,move)
            ret.append(nod)
        return ret
    
def minutil(game:xo,nod:node):
    if game.terminal_state(nod):
        return nod.utility
    min_utility=2
    children=nod.child_gen(game)
    for child in children:
        maxutil(game,child)
        if child.utility<min_utility:
            min_utility=child.utility
    nod.utility=min_utility
    
def maxutil(game:xo,nod:node):
    if game.terminal_state(nod):
        return nod.utility
    max_utility=-2
    children=nod.child_gen(game)
    for child in children:
        minutil(game,child)
        if child.utility>max_utility:
            max_utility=child.utility
    nod.utility=max_utility

def minmax(game:xo,nod:node):
    max_utility=-2
    max_node=None
    children=nod.child_gen(game)
    for child in children:
        minutil(game,child)
        if child.utility>max_utility:
            max_node=child
            max_utility=child.utility
    return max_node.action
    
print('''Board locations are as follows
1|2|3
-+-+-
4|5|6
-+-+-
7|8|9
''')
play_xo=xo()
play_xo.play_game()


Board locations are as follows
1|2|3
-+-+-
4|5|6
-+-+-
7|8|9

 | | 
-+-+-
 | | 
-+-+-
 | | 

enter a position for O player: 1
O| | 
-+-+-
 | | 
-+-+-
 | | 

O| | 
-+-+-
 |X| 
-+-+-
 | | 

enter a position for O player: 2
O|O| 
-+-+-
 |X| 
-+-+-
 | | 

O|O|X
-+-+-
 |X| 
-+-+-
 | | 

enter a position for O player: 6
O|O|X
-+-+-
 |X|O
-+-+-
 | | 

O|O|X
-+-+-
 |X|O
-+-+-
X| | 

X player wins


(b) Modify the previous answer to calculate in each game the following:

* the maximum depth of exploration of the game tree in a game, and
* number of leaves of the game tree whose scores were computed be the end of the game.




In [15]:
class xo:
    def __init__(self):
        self.grid=((' ',' ',' '),(' ',' ',' '),(' ',' ',' '))
        self.player_turn='O'
        self.utility=0
        self.num_leaves_explored=0
        self.num_turns_played=0
        self.max_depth=0

    def moves(self,nod):
        return [i for i in range(1,10) if nod[(i-1)//3][(i-1)%3]==' ']

    def result(self,nod,action):
        if nod.player_turn=='O':
            temp=[]
            for row in nod.state:
                temp.append(list(row))
            temp[(action-1)//3][(action-1)%3]='O'
            temp1=[]
            for row in temp:
                temp1.append(tuple(row))
            return tuple(temp1)
        
        temp=[]
        for row in nod.state:
            temp.append(list(row))
        temp[(action-1)//3][(action-1)%3]='X'
        temp1=[]
        for row in temp:
            temp1.append(tuple(row))
        return tuple(temp1)
                
    def o_play(self,index): 
        temp=[]
        for row in self.grid:
            temp.append(list(row))
        if(temp[(index-1)//3][(index-1)%3]==' '):
            temp[(index-1)//3][(index-1)%3]='O'
        else:
            self.o_play(int(input("wrong position! please enter another position: ")))
            return
        temp1=[]
        for row in temp:
            temp1.append(tuple(row))
        self.grid=tuple(temp1)
        self.player_turn='X'
        self.num_turns_played+=1

    def terminal_state(self,nod):
        if nod.player_turn=='O':
            for i in range(3):
                for j in range(3):
                    if not nod.state[i][j]=='X':
                        break
                else:
                    nod.utility=1
                    return 1
            
            for i in range(3):
                for j in range(3):
                    if not nod.state[j][i]=='X':
                        break
                else:
                    nod.utility=1
                    return 1
                
            for i in range(3):
                if not nod.state[i][i]=='X':
                    break
            else:
                nod.utility=1
                return 1
                                                                     
            for i in range(3):
                if not nod.state[i][2-i]=='X':
                    break
            else:
                nod.utility=1
                return 1
            
            return 0
                                                                     
        for i in range(3):
            for j in range(3):
                if not nod.state[i][j]=='O':
                    break
            else:
                nod.utility=-1
                return 2
        
        for i in range(3):
            for j in range(3):
                if not nod.state[j][i]=='O':
                    break
            else:
                nod.utility=-1
                return 2
            
        for i in range(3):
            if not nod.state[i][i]=='O':
                break
        else:
            nod.utility=-1
            return 2
                                                                    
        for i in range(3):
            if not nod.state[i][2-i]=='O':
                break
        else:
            nod.utility=-1
            return 2
        
        if nod.num_turns_played==9:
            return 3
        return 0
        
    def display(self):
        for i in range(2):
            for j in range(2):
                print(f"{self.grid[i][j]}|",end='')
            print(self.grid[i][2])
            print("-+-+-")
        print(f'{self.grid[2][0]}|{self.grid[2][1]}|{self.grid[2][2]}\n')
                                                                     
    def x_play(self):
        index=minmax(self,node(self.grid,self.num_turns_played,self.player_turn))
        temp=[]
        for row in self.grid:
            temp.append(list(row))
        if(temp[(index-1)//3][(index-1)%3]==' '):
            temp[(index-1)//3][(index-1)%3]='X'
        else:
            self.o_play(int(input("wrong position! please enter another position: ")))
        temp1=[]
        for row in temp:
            temp1.append(tuple(row))
        self.grid=tuple(temp1)
        self.player_turn='O'
        self.num_turns_played+=1

    def play_game(self):
        terminal_state=self.terminal_state(node(self.grid,self.num_turns_played,self.player_turn))
        while not terminal_state:
            self.display()
            if self.player_turn=='O':
                self.o_play(int(input("enter a position for O player: ")))
            else:
                self.x_play()
            terminal_state=self.terminal_state(node(self.grid,self.num_turns_played,self.player_turn))
        self.display()
        if terminal_state==1:
            print("X player wins")
            
        elif terminal_state==2:
            print("O player wins")
            
        else:
            print("Game ended in a Draw")
            
class node:
    def __init__(self,state,num_turns,to_play,action=None,utility=0):
        self.state=state
        self.utility=utility
        self.num_turns_played=num_turns    
        self.player_turn=to_play
        self.action=action

    def child_gen(self,game:xo):
        ret=[]
        for move in game.moves(self.state):
            if self.player_turn=='X':
                z='O'
            else:
                z='X'
            nod=node(game.result(self,move),self.num_turns_played+1,z,move)
            ret.append(nod)
        return ret
    
def minutil(game:xo,nod:node,depth):
    if nod.num_turns_played>depth[0]:
        depth[0]=nod.num_turns_played
    if game.terminal_state(nod):
        game.num_leaves_explored+=1
        return
    min_utility=2
    children=nod.child_gen(game)
    for child in children:
        maxutil(game,child,depth)
        if child.utility<min_utility:
            min_utility=child.utility
    nod.utility=min_utility
    
def maxutil(game:xo,nod:node,depth):
    if nod.num_turns_played>depth[0]:
        depth[0]=nod.num_turns_played
    if game.terminal_state(nod):
        game.num_leaves_explored+=1
        return
    max_utility=-2
    children=nod.child_gen(game)
    for child in children:
        minutil(game,child,depth)
        if child.utility>max_utility:
            max_utility=child.utility
    nod.utility=max_utility

def minmax(game:xo,nod:node):
    depth=[0]
    max_utility=-2
    max_node=None
    children=nod.child_gen(game)
    for child in children:
        minutil(game,child,depth)
        if child.utility>max_utility:
            max_node=child
            max_utility=child.utility
    if depth[0]-nod.num_turns_played+1>game.max_depth:
        game.max_depth=depth[0]-nod.num_turns_played
    return max_node.action
    
print('''Board locations are as follows
1|2|3
-+-+-
4|5|6
-+-+-
7|8|9
''')
play_xo=xo()
play_xo.play_game()
print(f"no. of leaves explored: {play_xo.num_leaves_explored} and max depth: {play_xo.max_depth}")

Board locations are as follows
1|2|3
-+-+-
4|5|6
-+-+-
7|8|9

 | | 
-+-+-
 | | 
-+-+-
 | | 

enter a position for O player: 3
 | |O
-+-+-
 | | 
-+-+-
 | | 

 | |O
-+-+-
 |X| 
-+-+-
 | | 

enter a position for O player: 9
 | |O
-+-+-
 |X| 
-+-+-
 | |O

 | |O
-+-+-
 |X|X
-+-+-
 | |O

enter a position for O player: 4
 | |O
-+-+-
O|X|X
-+-+-
 | |O

X| |O
-+-+-
O|X|X
-+-+-
 | |O

enter a position for O player: 2
X|O|O
-+-+-
O|X|X
-+-+-
 | |O

X|O|O
-+-+-
O|X|X
-+-+-
X| |O

enter a position for O player: 8
X|O|O
-+-+-
O|X|X
-+-+-
X|O|O

Game ended in a Draw
no. of leaves explored: 28199 and max depth: 8


(c) Implement alpha-beta pruning minimax search to solve the Tic-Tac-Toe and repeat part (b). 

Compare your results with vanilla minimax search and see on which all parameters part (b) is alpha-bet pruning search better. Also obtain by what factor is it better.

In [16]:
class xo:
    def __init__(self):
        self.grid=((' ',' ',' '),(' ',' ',' '),(' ',' ',' '))
        self.player_turn='O'
        self.utility=0
        self.num_leaves_explored=0
        self.num_turns_played=0
        self.max_depth=0

    def moves(self,nod):
        return [i for i in range(1,10) if nod[(i-1)//3][(i-1)%3]==' ']

    def result(self,nod,action):
        if nod.player_turn=='O':
            temp=[]
            for row in nod.state:
                temp.append(list(row))
            temp[(action-1)//3][(action-1)%3]='O'
            temp1=[]
            for row in temp:
                temp1.append(tuple(row))
            return tuple(temp1)
        
        temp=[]
        for row in nod.state:
            temp.append(list(row))
        temp[(action-1)//3][(action-1)%3]='X'
        temp1=[]
        for row in temp:
            temp1.append(tuple(row))
        return tuple(temp1)
                
    def o_play(self,index): 
        temp=[]
        for row in self.grid:
            temp.append(list(row))
        if(temp[(index-1)//3][(index-1)%3]==' '):
            temp[(index-1)//3][(index-1)%3]='O'
        else:
            self.o_play(int(input("wrong position! please enter another position: ")))
            return
        temp1=[]
        for row in temp:
            temp1.append(tuple(row))
        self.grid=tuple(temp1)
        self.player_turn='X'
        self.num_turns_played+=1

    def terminal_state(self,nod):
        if nod.player_turn=='O':
            for i in range(3):
                for j in range(3):
                    if not nod.state[i][j]=='X':
                        break
                else:
                    nod.utility=1
                    return 1
            
            for i in range(3):
                for j in range(3):
                    if not nod.state[j][i]=='X':
                        break
                else:
                    nod.utility=1
                    return 1
                
            for i in range(3):
                if not nod.state[i][i]=='X':
                    break
            else:
                nod.utility=1
                return 1
                                                                     
            for i in range(3):
                if not nod.state[i][2-i]=='X':
                    break
            else:
                nod.utility=1
                return 1
            
            return 0
                                                                     
        for i in range(3):
            for j in range(3):
                if not nod.state[i][j]=='O':
                    break
            else:
                nod.utility=-1
                return 2
        
        for i in range(3):
            for j in range(3):
                if not nod.state[j][i]=='O':
                    break
            else:
                nod.utility=-1
                return 2
            
        for i in range(3):
            if not nod.state[i][i]=='O':
                break
        else:
            nod.utility=-1
            return 2
                                                                    
        for i in range(3):
            if not nod.state[i][2-i]=='O':
                break
        else:
            nod.utility=-1
            return 2
        
        if nod.num_turns_played==9:
            return 3
        return 0
        
    def display(self):
        for i in range(2):
            for j in range(2):
                print(f"{self.grid[i][j]}|",end='')
            print(self.grid[i][2])
            print("-+-+-")
        print(f'{self.grid[2][0]}|{self.grid[2][1]}|{self.grid[2][2]}\n')
                                                                     
    def x_play(self):
        index=alpha_beta_minimax(self,node(self.grid,self.num_turns_played,self.player_turn))
        temp=[]
        for row in self.grid:
            temp.append(list(row))
        if(temp[(index-1)//3][(index-1)%3]==' '):
            temp[(index-1)//3][(index-1)%3]='X'
        else:
            self.o_play(int(input("wrong position! please enter another position: ")))
        temp1=[]
        for row in temp:
            temp1.append(tuple(row))
        self.grid=tuple(temp1)
        self.player_turn='O'
        self.num_turns_played+=1

    def play_game(self):
        terminal_state=self.terminal_state(node(self.grid,self.num_turns_played,self.player_turn))
        while not terminal_state:
            self.display()
            if self.player_turn=='O':
                self.o_play(int(input("enter a position for O player: ")))
            else:
                self.x_play()
            terminal_state=self.terminal_state(node(self.grid,self.num_turns_played,self.player_turn))
        self.display()
        if terminal_state==1:
            print("X player wins")
            
        elif terminal_state==2:
            print("O player wins")
            
        else:
            print("Game ended in a Draw")
        
class node:
    def __init__(self,state,num_turns,to_play,action=None,utility=0):
        self.state=state
        self.utility=utility
        self.num_turns_played=num_turns    
        self.player_turn=to_play
        self.action=action

    def child_gen(self,game:xo):
        ret=[]
        for move in game.moves(self.state):
            if self.player_turn=='X':
                z='O'
            else:
                z='X'
            nod=node(game.result(self,move),self.num_turns_played+1,z,move)
            ret.append(nod)
        return ret
    
def minutil(game:xo,nod:node,depth,alpha,beta):
    if nod.num_turns_played>depth[0]:
        depth[0]=nod.num_turns_played
    if game.terminal_state(nod):
        game.num_leaves_explored+=1
        return
    min_utility=2
    children=nod.child_gen(game)
    for child in children:
        maxutil(game,child,depth,alpha,beta)
        if child.utility<min_utility:
            min_utility=child.utility
            if min_utility<beta:
                 beta=min_utility
        if child.utility<alpha:
            break 
    nod.utility=min_utility
    
def maxutil(game:xo,nod:node,depth,alpha,beta):
    if nod.num_turns_played>depth[0]:
        depth[0]=nod.num_turns_played
    if game.terminal_state(nod):
        game.num_leaves_explored+=1
        return
    max_utility=-2
    children=nod.child_gen(game)
    for child in children:
        minutil(game,child,depth,alpha,beta)
        if child.utility>max_utility:
            max_utility=child.utility
            if max_utility>alpha:
                alpha=max_utility
        if child.utility>beta:
            break
    nod.utility=max_utility

def alpha_beta_minimax(game:xo,nod:node):
    alpha=-1
    beta=1
    depth=[0]
    max_utility=-2
    max_node=None
    children=nod.child_gen(game)
    for child in children:
        minutil(game,child,depth,alpha,beta)
        if child.utility>max_utility:
            max_node=child
            max_utility=child.utility
            if max_utility>alpha:
                alpha=max_utility
    if depth[0]-nod.num_turns_played>game.max_depth:
        game.max_depth=depth[0]-nod.num_turns_played
    return max_node.action
    
print('''Board locations are as follows
1|2|3
-+-+-
4|5|6
-+-+-
7|8|9
''')
play_xo=xo()
play_xo.play_game()
print(f"no. of leaves explored: {play_xo.num_leaves_explored} and max depth: {play_xo.max_depth}")

Board locations are as follows
1|2|3
-+-+-
4|5|6
-+-+-
7|8|9

 | | 
-+-+-
 | | 
-+-+-
 | | 

enter a position for O player: 1
O| | 
-+-+-
 | | 
-+-+-
 | | 

O| | 
-+-+-
 |X| 
-+-+-
 | | 

enter a position for O player: 2
O|O| 
-+-+-
 |X| 
-+-+-
 | | 

O|O|X
-+-+-
 |X| 
-+-+-
 | | 

enter a position for O player: 7
O|O|X
-+-+-
 |X| 
-+-+-
O| | 

O|O|X
-+-+-
X|X| 
-+-+-
O| | 

enter a position for O player: 6
O|O|X
-+-+-
X|X|O
-+-+-
O| | 

O|O|X
-+-+-
X|X|O
-+-+-
O|X| 

enter a position for O player: 9
O|O|X
-+-+-
X|X|O
-+-+-
O|X|O

Game ended in a Draw
no. of leaves explored: 7013 and max depth: 8
